# Импортируем нужные библиотеки

In [ ]:
from gigachat import GigaChat
from langchain_core.prompts import PromptTemplate
import os
from getpass import getpass
import feedparser
import pandas as pd
from datetime import datetime
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics import silhouette_score

# Парсинг

Основная логика парсинга:

1) делаем запрос к ArXiv API  используя параметры поиска, я выбрал  ключевые слова ("video generation", "video synthesis") и категории (cs.CV, cs.AI). Для пагинации  используется параметр , который увеличивается на размер страницы после каждого запроса. Это нужно чтобы предотвратить дублирование и получить большее количество статей.

2) Далее фильтруем статьи по году и загружаем их метаданные в датафрейм для дальнейшего использования.


Я собираю не только метаданные, но и сами тексты аннотаций, которые необходимы для последующего использования.


In [ ]:
def fetch_arxiv_metadata_with_pagination(query, max_results_per_request=500):
    all_articles = []
    start = 0
    step = max_results_per_request

    while True:
        url = f"http://export.arxiv.org/api/query?search_query={query}&start={start}&max_results={step}"
        feed = feedparser.parse(url)

        articles = []
        for entry in feed.entries:
            published = datetime.strptime(entry.published, "%Y-%m-%dT%H:%M:%SZ")
            published_year = published.year

            if published_year != 2024:
                continue

            article_data = {
                'title': entry.title,
                'annotation': entry.summary,
                'year': published,
                'categories': [tag['term'] for tag in entry.tags],
                'url': entry.link
            }
            articles.append(article_data)

        all_articles.extend(articles)

        if len(feed.entries) < step:
            print("Достигнут конец списка статей.")
            break

        start += step

    df = pd.DataFrame(all_articles)
    return df

query = "video+generation+OR+video+synthesis+OR+generative+video+OR+text-to-video+cat:cs.CV+OR+cat:cs.AI+submittedDate:[20240101+TO+20241231]"
meta_data = fetch_arxiv_metadata_with_pagination(query, max_results_per_request=500)


Достигнут конец списка статей.


Из большого множества парсеров я выбрал  feedparser, потому что API ArXiv возвращает данные в формате Atom/RSS, который этот пакет прекрасно обрабатывает. Этот подход также упрощает работу с пагинацией, поскольку нам нужно последовательно запрашивать данные небольшими частями.

In [ ]:
len(meta_data)

1258

In [ ]:
meta_data.head()

,title,annotation,year,categories,url
0,Fine-gained Zero-shot Video Sampling,Incorporating a temporal dimension into pretra...,2024-07-31 09:36:58,"[cs.CV, cs.AI]",http://arxiv.org/abs/2407.21475v1
1,Factorized-Dreamer: Training A High-Quality Vi...,Text-to-video (T2V) generation has gained sign...,2024-08-19 16:08:00,"[cs.CV, cs.AI]",http://arxiv.org/abs/2408.10119v1
2,SpatialDreamer: Self-supervised Stereo Video S...,Stereo video synthesis from a monocular input ...,2024-11-18 15:12:59,"[cs.CV, cs.AI]",http://arxiv.org/abs/2411.11934v1
3,Snap Video: Scaled Spatiotemporal Transformers...,Contemporary models for generating images show...,2024-02-22 18:55:08,"[cs.CV, cs.AI]",http://arxiv.org/abs/2402.14797v1
4,SNED: Superposition Network Architecture Searc...,While AI-generated content has garnered signif...,2024-05-31 21:12:30,"[cs.CV, cs.AI]",http://arxiv.org/abs/2406.00195v1


# Выделение трендов

Итак с помощью пагинации нам удалось собрать 1258 методанных статей и их аннотации  по теме видеогенерация в 2024 году.
Далее я буду вычленять из них ключевые слова (исключая стоп слова), разбивать их по темам и выделять тренды, для этого я буду использовать tf-idf и кластеризацию k-средних, параметр k буду подбирать на основе silhouette_score для выделения оптимального количества трендов.

In [ ]:
nlp = spacy.load("en_core_web_sm")

custom_stop_words = ['video', 'generation', 'model', 'models', 'using', 'based']

In [ ]:
def extract_keywords(text):
    doc = nlp(text)
    keywords = [token.text.lower() for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB'] and not token.is_stop and token.text.lower() not in custom_stop_words]
    noun_phrases = [chunk.text.lower() for chunk in doc.noun_chunks if len(chunk.text.split()) > 1 and chunk.text.lower() not in custom_stop_words]
    return keywords + noun_phrases

In [ ]:
all_keywords = []
for abstract in meta_data['annotation']:
    keywords = extract_keywords(abstract)
    all_keywords.extend(keywords)

keyword_counts = Counter(all_keywords)

top_keywords = [word for word, count in keyword_counts.most_common(100)]

vectorizer = TfidfVectorizer(vocabulary=top_keywords, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(meta_data['annotation'])

In [ ]:
silhouette_scores = []
for k in range(3, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(tfidf_matrix)
    score = silhouette_score(tfidf_matrix, clusters)
    silhouette_scores.append((k, score))

best_k = max(silhouette_scores, key=lambda x: x[1])[0]
print(f"best_k: {best_k}")

best_k: 7


После кластеризации я буду использовать llm (gigachat) для генерации на основе ключевых слов связанного тренда.

In [ ]:
llm = GigaChat(
    credentials=getpass('INSERT_AUTH.KEY:'),
    model="GigaChat-Max",
    verify_ssl_certs=False)

template = """
Твоя задача — описать тренд видеогенерации  на русском языке в виде короткого связного предложения с использованием ключевых слов.
Предложение должно быть четким, длиной ровно 5–7 слов, без лишних слов.
Используйте 2–3 ключевых слова из списка, связав их логически.

Пример хорошего тренда: Генерация 3D-видео по аудиомоделям
Пример плохого тренда: Генерация видео 3D аудиомодели

Ключевые слова: {keywords}

Сформулируйте тренд:
"""

In [ ]:


num_clusters = best_k
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(tfidf_matrix)

meta_data['cluster'] = clusters

def get_cluster_keywords(cluster_id):
    cluster_abstracts = meta_data[meta_data['cluster'] == cluster_id]['annotation']
    cluster_keywords = []
    for abstract in cluster_abstracts:
        keywords = extract_keywords(abstract)
        cluster_keywords.extend(keywords)
    return Counter(cluster_keywords).most_common(10)

trends = []
for i in range(num_clusters):
    top_words = get_cluster_keywords(i)
    keywords = ', '.join(word for word, _ in top_words[:5])

    prompt = PromptTemplate(input_variables=['keywords'], template=template).format(keywords=keywords)
    response = llm.chat(prompt).choices[0].message.content

    word_count = len(response.split())
    trends.append(response)

result = pd.DataFrame({'Trends': trends})





# Результат

In [ ]:
for i, trend in enumerate(result['Trends']):
    print(f'Тренд №{i + 1}: {trend}')

Тренд №1: Генерация временных видеофреймов на основе данных
Тренд №2: Улучшение качества видеодиффузии по текстовым и визуальным данным.
Тренд №3: Генерация видео на основе понимания языка
Тренд №4: Детекция объектов в видеороликах.
Тренд №5: Генерация видео с говорящими визуальными моделями
Тренд №6: Текстовая генерация видео с контролем движения
Тренд №7: Генерация 3D-сцен с мультивидами


Видим, что тренды получились весьма обоснованными и действительно отражают актуальные направления в области видеогенерации и анализа видео.